In [ ]:
from py2neo import Graph

neo_host = "neo4j"
neo_port = 7687

graph = Graph(f"bolt://{neo_host}:{neo_port}")

In [ ]:
cursor = graph.run("""MATCH ()-[r:Dissed]->(target)
                        RETURN target, count(r) AS num
                        ORDER BY num DESC
                    """)

nb_targets = 3
most_dissed_targets = []
for i in range(nb_targets):
    if cursor.forward():
        most_dissed_targets.append(cursor.current['target']['name'])

most_dissed_targets

In [ ]:
cursor = graph.run("""MATCH (artist)-[r:Dissed]->()
                        RETURN artist, count(r) AS num
                        ORDER BY num DESC
                    """)

nb_targets = 3
most_dissors_artists = []
for i in range(nb_targets):
    if cursor.forward():
        most_dissors_artists.append(cursor.current['artist']['name'])

most_dissors_artists